In [1]:
!python --version

Python 3.11.11


In [2]:
!git clone https://github.com/facebookresearch/LayerSkip.git

Cloning into 'LayerSkip'...
remote: Enumerating objects: 279, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 279 (delta 122), reused 116 (delta 89), pack-reused 101 (from 1)
Receiving objects: 100% (279/279), 11.55 MiB | 35.62 MiB/s, done.
Resolving deltas: 100% (153/153), done.


In [3]:
import os
os.getcwd()

'/content'

In [4]:
%cd '/content/LayerSkip'

/content/LayerSkip


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install torchaudio

In [ ]:
!pip uninstall transformers
!pip install transformers==4.45.0

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `Llama` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Llama`


In [9]:
import torch
import torchaudio
import torchvision

print(f"Torch Version: {torch.__version__}")
print(f"Torchaudio Version: {torchaudio.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")


Torch Version: 2.5.1+cu124
Torchaudio Version: 2.5.1+cu124
Torchvision Version: 0.20.1+cu124


In [21]:
! torchrun benchmark.py --model facebook/layerskip-llama3.2-1B \
    --dataset mmlu \
    --num_samples 100 \
    --generation_strategy self_speculative \
    --exit_layer 8 \
    --num_speculations 6 \
    --output_dir ./logs

Optimizing generation config for MMLU...
Updated generation config: max_steps=20, temperature=0.3
Benchmarking on MMLU with 100 samples...
Benchmarking MMLU:   0% 0/100 [00:00<?, ?it/s]Type:  <class 'data.EvaluationExample'>
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.
2025-03-05 21:27:54.039228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741210074.060687   24914 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741210074.068733   24914 cuda_b

## Baseline Results

! torchrun benchmark.py --model facebook/layerskip-llama3.2-1B \
    --dataset cnn_dm_summarization \
    --num_samples 100 \
    --generation_strategy self_speculative \
    --exit_layer 8 \
    --num_speculations 6 \
    --output_dir ./logs

{'predicted_text': {'rouge-l': 0.09527777880430222, 'rouge-1': 0.1348915547132492, 'rouge-2': 0.05371306464076042, 'rouge-3': 0.027781713753938675, 'bleu_score': 0.0, 'exact_match': 2145.550048828125}, 'acceptance_rate': {'mean': 0.7760123947262764}, 'total_time': {'mean': 11.370401830673218}, 'time_per_token': {'mean': 0.02220781607553363}, 'tokens_per_second': {'mean': 46.37881397247315}}
